# Analysis

This notebook presents execution and results from:

* Base case analysis
* Scenario analysis
* Sensitivity analysis

Credit:

* Analysis of the spread of replication results was adapted from Tom Monks (2024) HPDM097 - Making a difference with health data (https://github.com/health-data-science-OR/stochastic_systems) (MIT License).

In [1]:
from model import Defaults, Trial
import itertools
from IPython.display import display
import pandas as pd
import plotly.express as px

## Spread of replication results

In [6]:
# TODO: Provide a range of examples visualising results, ensuring we're always incorporating uncertainty

In [2]:
def plot_results_spread(column, x_label, y_label='Frequency'):
    """
    Plot spread of results from across replications, for chosen column.

    Arguments:
        column (str):
            Name of column to plot.
        x_label (str):
            X axis label.
        y_label (str):
            Y axis label
    """
    fig = px.histogram(trial.trial_results_df[column])
    fig.update_layout(
        xaxis_title=x_label,
        yaxis_title=y_label
    )
    fig.show()


param = Defaults()
trial = Trial(param)
trial.run_trial()

plot_results_spread('arrivals', 'Arrivals')
plot_results_spread('mean_q_time_nurse', 'Mean wait time for nurse')
plot_results_spread('mean_time_with_nurse', 'Mean length of nurse consultation')
plot_results_spread('mean_nurse_utilisation', 'Mean nurse utilisation')

In [3]:
# TODO: Create Latex-formatted table.

## Scenario analysis

In [4]:
# Define a set of scenarios
scenarios = {
    'patient_inter': [3, 4, 5],
    'mean_n_consult_time': [8, 10, 12],
    'number_of_nurses': [4, 5, 6]
}

# Find every possible permutation of the scenarios
all_scenarios_tuples = list(itertools.product(*scenarios.values()))
# Convert back into dictionaries
all_scenarios_dicts = [
    dict(zip(scenarios.keys(), p)) for p in all_scenarios_tuples]
# Preview some of the scenarios
print(f'There are {len(all_scenarios_dicts)} scenarios. Running:')

# Run the scenarios...
results = []
for index, scenario_to_run in enumerate(all_scenarios_dicts):
    print(scenario_to_run)

    # Overwrite defaults from the passed dictionary
    param = Defaults()
    param.scenario_name = index
    for key in scenario_to_run:
        setattr(param, key, scenario_to_run[key])

    # Run trial and keep trial-level results
    my_trial = Trial(param)
    my_trial.run_trial()
    results.append(my_trial.trial_results_df)

# View mean results by scenario
display(pd.concat(results)
        .drop('run_number', axis=1)
        .groupby('scenario')
        .mean()
        .head(20))

There are 27 scenarios. Running:
{'patient_inter': 3, 'mean_n_consult_time': 8, 'number_of_nurses': 4}
{'patient_inter': 3, 'mean_n_consult_time': 8, 'number_of_nurses': 5}
{'patient_inter': 3, 'mean_n_consult_time': 8, 'number_of_nurses': 6}
{'patient_inter': 3, 'mean_n_consult_time': 10, 'number_of_nurses': 4}
{'patient_inter': 3, 'mean_n_consult_time': 10, 'number_of_nurses': 5}
{'patient_inter': 3, 'mean_n_consult_time': 10, 'number_of_nurses': 6}
{'patient_inter': 3, 'mean_n_consult_time': 12, 'number_of_nurses': 4}
{'patient_inter': 3, 'mean_n_consult_time': 12, 'number_of_nurses': 5}
{'patient_inter': 3, 'mean_n_consult_time': 12, 'number_of_nurses': 6}
{'patient_inter': 4, 'mean_n_consult_time': 8, 'number_of_nurses': 4}
{'patient_inter': 4, 'mean_n_consult_time': 8, 'number_of_nurses': 5}
{'patient_inter': 4, 'mean_n_consult_time': 8, 'number_of_nurses': 6}
{'patient_inter': 4, 'mean_n_consult_time': 10, 'number_of_nurses': 4}
{'patient_inter': 4, 'mean_n_consult_time': 10, 'n

,arrivals,mean_q_time_nurse,mean_time_with_nurse,mean_nurse_utilisation
scenario,,,,
0,14361.677419,2.148602,7.981360,0.663200
1,14361.677419,0.521858,7.981302,0.530563
2,14361.677419,0.137471,7.981267,0.442141
3,14361.677419,9.071256,9.976640,0.828867
4,14361.677419,1.846286,9.976662,0.663165
5,14361.677419,0.520991,9.976627,0.552643
6,14361.677419,313.277764,11.972974,0.992260
7,14361.677419,6.150912,11.971870,0.795700
8,14361.677419,1.603622,11.971937,0.663135


## Sensitivity analysis

In [5]:
# TODO